In [ ]:
# Assigment 1 forwad pass and backpropagation without using any library

x = [1,-1]
t = [1,0]

w = [[1,1,1][-1,-1,-1]]
b = [0,0,0]

v = [[1,1][-1,-1][-1,-1]]
c = [0,0]


# forward pass

print(f"x: {x}")

# first layer
print(f"W: {w}")

k = [sum([x[i]*w[i][j] for i in range(len(x))]) + b[j] for j in range(len(b))]

print(f"k: {k}")

# sigmoid layer
z = [1/(1+math.exp(-k[i])) for i in range(len(k))]

print(f"z: {z}")

# second layer
print(f"V: {v}")

o = [sum([z[i]*v[i][j] for i in range(len(z))]) + c[j] for j in range(len(c))]

print(f"o: {o}")

# softmax layer
y = [math.exp(o[i])/sum([math.exp(o[j]) for j in range(len(o))]) for i in range(len(o))]

print(f"y: {y}")

# cross entropy loss
loss = -sum([t[i]*math.log(y[i]) for i in range(len(t))])

print(f"loss: {loss}")
